In [5]:
import datetime as dt
from utility import (
    read_sql,
    fd_basicinfo,
    fd_typeclass,
    fd_hshkiport,
    fd_assetportfolio,
    nearest_report_date
)
import pandas as pd

In [2]:
# 获取回溯的报告日

today = dt.datetime.today()
date_3yrs_ago = today.replace(year=today.year - 3)
report_dates_begin = nearest_report_date(date_3yrs_ago.strftime("%Y%m%d"))
current_date = today.strftime("%Y%m%d")

In [3]:
# 获取所有目标股票型基金（清算结束日期晚于当前日）

security_ids = read_sql(f"""
select SECURITYID from TQ_FD_TYPECLASS 
WHERE
    ISVALID = 1 AND
    L1CODE = 1 AND
    (ENDDATE>='{today}' or ENDDATE = '19000101') 
ORDER BY SECURITYID;
""")["SECURITYID"].unique().tolist()

In [13]:
basic_info = fd_basicinfo(security_ids, current_date)
type_info = fd_typeclass(security_ids, current_date)
hkport_info = fd_hshkiport(security_ids, report_dates_begin)
hkport_info = hkport_info[hkport_info.INDUSTRYNAME == "合计"].groupby(["SECURITYID"], as_index=False)[["ACCNETMKTCAP"]].mean()
asset_port = fd_assetportfolio(security_ids, report_dates_begin).groupby("SECURITYID", as_index=False)[["EQUITYINVERTO"]].mean()

In [19]:
df = pd.merge(basic_info, type_info, on="SECURITYID")
df = pd.merge(df, asset_port, on="SECURITYID", how="left")
df = pd.merge(df, hkport_info, on="SECURITYID", how="left")
df["ACCNETMKTCAP"] = df["ACCNETMKTCAP"].fillna(0)

In [23]:
# 分类
df.loc[(df["L3NAME"].isin(["其他行业股票型基金", "偏股型基金（股票上下限60%-95%）", '标准股票型基金', '医药行业股票型基金', '封闭式标准股票型基金'])), "股票型（子类）"] = "普通股票型基金"
df.loc[(df["L3NAME"].isin(["标准指数股票型基金"])), "股票型（子类）"] = "被动指数型基金"
df.loc[(df["L3NAME"].isin(["增强指数股票型基金"])), "股票型（子类）"] = "增强指数型基金"
df.loc[(df["ACCNETMKTCAP"] / df["EQUITYINVERTO"] > 0.5), "股票型（子类）"] = "港股通股票型基金"

In [24]:
df.groupby("股票型（子类）").count()

,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,EQUITYINVERTO,ACCNETMKTCAP
股票型（子类）,,,,,,,,,,,,,,
增强指数型基金,63,63,45,63,63,63,63,63,63,63,63,63,60,63
普通股票型基金,207,207,60,207,207,207,207,207,207,207,207,207,206,207
港股通股票型基金,12,12,8,12,12,12,12,12,12,12,12,12,12,12
被动指数型基金,342,342,213,342,342,342,342,342,342,342,342,342,329,342


In [25]:
df

,SECURITYID,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,EQUITYINVERTO,ACCNETMKTCAP,股票型（子类）
0,1030000030,国泰沪深300指数证券投资基金,国泰沪深300指数,020011,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,93.975000,0.0,被动指数型基金
1,1030000034,华安MSCI中国A股指数增强型证券投资基金,None,040002,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.2,增强指数股票型基金,94.049167,0.0,增强指数型基金
2,1030000045,博时裕富沪深300指数证券投资基金,博时沪深300指数,050002,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,93.689167,0.0,被动指数型基金
3,1030000045,博时裕富沪深300指数证券投资基金,博时沪深300指数,050002,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,93.689167,0.0,被动指数型基金
4,1030000096,易方达上证50指数增强型证券投资基金,易方达上证50增强,110003,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.2,增强指数股票型基金,92.823333,0.0,增强指数型基金
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,1030005100,信诚中证智能家居指数分级证券投资基金,信诚中证智能家居指数分级,165524,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,94.028889,0.0,被动指数型基金
934,1030001329,信诚沪深300指数分级证券投资基金,信诚沪深300指数分级,165515,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,94.640000,0.0,被动指数型基金
935,1030002379,信诚中证800医药指数分级证券投资基金,信诚中证800医药指数分级,165519,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,94.184444,0.0,被动指数型基金
936,1030001071,信诚中证500指数分级证券投资基金,信诚中证500指数,165511,证券投资基金,指数型,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,93.046667,0.0,被动指数型基金
